In [ ]:
# Data wrangling
import pandas as pd

# Deep learning 
import tensorflow as tf

# Loading date wrangling package
from datetime import datetime

# Array math
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# RNN layers 
from tensorflow.keras.layers import Dense, SimpleRNN 

# Model creation API 
from tensorflow.keras.models import Sequential

# Data standartization 
from sklearn.preprocessing import StandardScaler

In [ ]:
# Reading the input data
d = pd.read_csv('data/energy-consumption/DAYTON_hourly.csv')

# Formating to datetime
d['Datetime'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in d['Datetime']]

# Making sure there are no duplicated data
# If there are some duplicates we average the data during those duplicated days
d = d.groupby('Datetime', as_index=False)['DAYTON_MW'].mean()

# Sorting the values
d.sort_values('Datetime', inplace=True)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(d['Datetime'], d['DAYTON_MW'])

In [ ]:
def create_X_Y(ts: list, lag: int) -> tuple:
    """
    A method to create X and Y matrix from a time series list for the training of 
    deep learning models 
    """
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag):
            Y.append(ts[i + lag])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an LSTM input shape 
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    return X, Y

In [ ]:
# Scaling the data 
scaler = StandardScaler()
scaler.fit(d['DAYTON_MW'].values.reshape(-1, 1))

# Transforming the data 
d['DAYTON_MW_scaled'] = scaler.transform(d['DAYTON_MW'].values.reshape(-1, 1))

In [ ]:
# Creating the dataset 
x, y = create_X_Y(d['DAYTON_MW_scaled'], lag=4)

In [ ]:
print(f"Shape of x: {x.shape}")
print(f"Shape of y: {y.shape}")

In [ ]:
# Defining a simple Rnn model
model = Sequential()
model.add(SimpleRNN(units=2, input_shape=(x.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Printing out the summary 
model.summary()

![](docs/simple-rnn.png)

In [ ]:
# Training the model 
history = model.fit(x, y, epochs=10, batch_size=64, verbose=1)

In [ ]:
# PLoting the loss
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()